In [1]:
import torchvision
from torchvision import transforms
import torch
from torch.utils.data import DataLoader

global maps

In [2]:
import pandas as pd
# import numpy as np
import os
from sklearn.utils import shuffle
from PIL import Image


data_root_dir = '/kaggle/input/classify-leaves'
data_csv_path = os.path.join(data_root_dir, './train.csv')

class Dataset(object):
    def __init__(self, data_frame, root_dir, transform=None) -> None:
        self.data_frame = data_frame
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)
    
    def __getitem__(self, idx):
        imgpath = os.path.join(self.root_dir, self.data_frame.iloc[idx,0])
        label = self.data_frame.iloc[idx,2]
        img = Image.open(imgpath).convert('RGB')
        if self.transform:
            img = self.transform(img)

        return img, label


def load_csv(path):
    df = pd.read_csv(path)
    list_type = df['label'].unique()
    dic = {}
    for i,t in enumerate(list_type):
        dic[t] = i
    global maps    
    maps = dic
    df['num_label'] = df['label'].map(dic)
    return shuffle(df)

def load_data(transform):
    data_df = load_csv(data_csv_path)
    train_dataset = Dataset(data_df.iloc[:14000], data_root_dir, transform)
    test_dataset = Dataset(data_df.iloc[14000:], data_root_dir, transform)

    return train_dataset, test_dataset

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
from torch import nn
from torch.nn import functional as F


class Residual(nn.Module):
    def __init__(self, in_channels, out_channels, use_1x1conv=False, stride=1) -> None:
        super().__init__()
        # conv2不会改变高宽,resnet的设计是当conv1的stride不等于1的时候会改变高宽，那么相应的会使用1x1conv去改变输入的高宽
        # conv1 k=3 s=2 p=1与conv3 k=1 s=2 p=0 的输出高宽相同(2p-k)是相同的
        # 在in_channels与out_channels不同的时候就会使用1x1conv
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, stride=stride)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride)
        else:
            self.conv3 = None

        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
    def forward(self ,x):
        y = F.relu(self.bn1(self.conv1(x)))
        y = self.bn2(self.conv2(y))
        if self.conv3:
            x = self.conv3(x)
        
        # 注意这里是直接相加，而不是叠加通道
        y += x
        return F.relu(y)

def resnet_block(in_channels, out_channels, num_residuals, first_block=False):
    blk = []
    for i in range(num_residuals):
        if i==0 and not first_block:
            # first_block的in_channels = out_channels
            blk.append(Residual(in_channels, out_channels, use_1x1conv=True, stride=2))
        else:
            blk.append(Residual(out_channels, out_channels))
    return blk


class ResNet18(nn.Module):
    def __init__(self, in_channels, dims=10) -> None:
        super().__init__()
        b1 = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64), nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2,padding=1)
        )
        b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))
        b3 = nn.Sequential(*resnet_block(64, 128, 2))
        b4 = nn.Sequential(*resnet_block(128, 256, 2))
        b5 = nn.Sequential(*resnet_block(256, 512, 2))
        self.net = nn.Sequential(
            b1, b2, b3, b4, b5,
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),
            nn.Linear(512, dims)
        )
    
    def forward(self, x):
        return self.net(x)

In [4]:
def ac(data_iter, net, device):
    num_acs = []
    for x, y in data_iter:
        x = x.to(device)
        y = y.to(device)
        y_hat = net(x)
        maxs, indexs = torch.max(y_hat, dim=1)
        num_acs.append(y.eq(indexs).sum()/indexs.shape[0])
    return sum(num_acs)/len(num_acs)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 64
lr = 0.1
epoch = 50

# 加载数据
trans = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor()])
set_train, set_test = load_data(trans)

iter_train = DataLoader(set_train, batch_size=batch_size,num_workers=4)
iter_test = DataLoader(set_test, batch_size=batch_size,num_workers=4)

net = ResNet18(3,176)
loss = nn.CrossEntropyLoss()
updater = torch.optim.SGD(net.parameters(), lr=lr)

In [6]:
# net.to(device)
# for i in range(epoch):
#     for batch_idx,(x,y) in enumerate(iter_train):
#         x = x.to(device)
#         y = y.to(device)

#         y_hat = net(x)
#         l = loss(y_hat, y)
#         updater.zero_grad()
#         l.backward()
#         updater.step()
#         # display
#         batch_idx += 1
#         if batch_idx%100 == 0:
#             test_accuracy = ac(iter_test, net, device)
#             train_accuracy = ac(iter_train, net, device)
#             print(f'epoch:{i} [{batch_idx*batch_size}/{len(set_train)}({100.0*batch_idx*batch_size/len(set_train):.2f}%)]\t',
#                 f'loss:{l.item():.2f}\t',
#                 f'train accuracy:{100.0*test_accuracy:.2f}%\t'
#                 f'test accuracy:{100.0*train_accuracy:.2f}%\t')
#     if (i+1)%10 == 0:
#         torch.save(net, 'resnet18_{}.pth'.format(i+1))

# 预测

In [7]:
maps_trans = {v:k for k,v in maps.items()}
net.eval()

ResNet18(
  (net): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (1): Sequential(
      (0): Residual(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): Residual(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_run

In [8]:
from torch.nn import functional as F

model_path = '/kaggle/input/train-out-model/resnet18_50.pth'
data_root_dir = '/kaggle/input/classify-leaves/'

net = torch.load(model_path)
test_df = pd.read_csv(os.path.join(data_root_dir,'test.csv'))

def predict(image_path, net):
    trans = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()])
    image_tensor = trans(Image.open(image_path).convert('RGB'))
    image_tensor = image_tensor.reshape((1,)+ image_tensor.shape).cuda()
#     return image_tensor.shape
    y_hat = F.softmax(net(image_tensor), dim=1)
    _, index = torch.max(y_hat, dim=1)
    return index.item()

In [9]:
test_df['label'] = None
for i in range(test_df.shape[0]):
    label = predict(os.path.join(data_root_dir,test_df.iloc[i,0]), net)
    label = maps_trans[label]
    test_df.iloc[i,1] = label
    if i%100==0:
        print(f'{i}/{test_df.shape[0]}({100.0*i/test_df.shape[0]:.2f})\t', f'label:{label}\t')

0/8800(0.00)	 label:crataegus_crus-galli	
100/8800(1.14)	 label:sassafras_albidum	
200/8800(2.27)	 label:sassafras_albidum	
300/8800(3.41)	 label:crataegus_crus-galli	
400/8800(4.55)	 label:crataegus_crus-galli	
500/8800(5.68)	 label:sassafras_albidum	
600/8800(6.82)	 label:sassafras_albidum	
700/8800(7.95)	 label:sassafras_albidum	
800/8800(9.09)	 label:crataegus_crus-galli	
900/8800(10.23)	 label:sassafras_albidum	
1000/8800(11.36)	 label:sassafras_albidum	
1100/8800(12.50)	 label:sassafras_albidum	
1200/8800(13.64)	 label:sassafras_albidum	
1300/8800(14.77)	 label:sassafras_albidum	
1400/8800(15.91)	 label:sassafras_albidum	
1500/8800(17.05)	 label:sassafras_albidum	
1600/8800(18.18)	 label:crataegus_crus-galli	
1700/8800(19.32)	 label:sassafras_albidum	
1800/8800(20.45)	 label:sassafras_albidum	
1900/8800(21.59)	 label:crataegus_crus-galli	
2000/8800(22.73)	 label:sassafras_albidum	
2100/8800(23.86)	 label:sassafras_albidum	
2200/8800(25.00)	 label:sassafras_albidum	
2300/8800(26.1

In [10]:
test_df.to_csv('submission.csv', index=False)
test_df['label'].value_counts()

label
sassafras_albidum        6538
crataegus_crus-galli     2119
pinus_strobus             103
ginkgo_biloba              23
quercus_muehlenbergii      13
staphylea_trifolia          4
Name: count, dtype: int64